In [1]:
from keras.datasets import cifar10
from keras.models import Model
from keras.layers import Dense, Dropout, SpatialDropout2D, Activation, Flatten
from keras.layers import Input, Convolution2D
from keras.layers.advanced_activations import LeakyReLU
from keras.regularizers import l2
from keras.layers.pooling import MaxPooling2D
from keras.optimizers import Adadelta
from keras.utils.np_utils import to_categorical

Using TensorFlow backend.


In [2]:
# Fetch data
(X_train, y_train), (X_test, y_test) = cifar10.load_data()

# Convert uint8 pixel values to float32 in the range [0, 1] (for sigmoid)
X_train = X_train.astype('float32')
X_train /= 255
X_test = X_test.astype('float32')
X_test /= 255

# Convert target values to categorical representation
y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)

# Verify data
print X_train.shape, y_train.shape
print X_test.shape, y_test.shape

(50000, 32, 32, 3) (50000, 10)
(10000, 32, 32, 3) (10000, 10)


In [3]:
# Training parameters
disc_optimizer = Adadelta(lr=1.0)
disc_regularizer = l2(1e-4)
disc_batch_size = 128
nb_epoch = 200

In [4]:
# Discriminator Model
# Based on https://github.com/fchollet/keras/blob/master/examples/cifar10_cnn.py

d_input = Input(shape=X_train.shape[1:])

d_layer = Convolution2D(32, 3, 3, border_mode='same', W_regularizer=disc_regularizer)(d_input)
d_layer = LeakyReLU()(d_layer)

d_layer = Convolution2D(32, 3, 3, border_mode='same')(d_layer)
d_layer = LeakyReLU()(d_layer)
d_layer = MaxPooling2D(pool_size=(2, 2))(d_layer) # 32 -> 16
d_layer = SpatialDropout2D(0.25)(d_layer)

d_layer = Convolution2D(64, 3, 3, border_mode='same', W_regularizer=disc_regularizer)(d_layer)
d_layer = LeakyReLU()(d_layer)

d_layer = Convolution2D(64, 3, 3, border_mode='same')(d_layer)
d_layer = LeakyReLU()(d_layer)
d_layer = MaxPooling2D(pool_size=(2, 2))(d_layer) # 16 -> 8
d_layer = SpatialDropout2D(0.25)(d_layer)

d_layer = Convolution2D(96, 3, 3, border_mode='same', W_regularizer=disc_regularizer)(d_layer)
d_layer = LeakyReLU()(d_layer)

d_layer = Convolution2D(96, 3, 3, border_mode='same')(d_layer)
d_layer = LeakyReLU()(d_layer)
d_layer = MaxPooling2D(pool_size=(2, 2))(d_layer) # 8 -> 4
d_layer = SpatialDropout2D(0.25)(d_layer)

d_layer = Convolution2D(128, 3, 3, border_mode='same', W_regularizer=disc_regularizer)(d_layer)
d_layer = LeakyReLU()(d_layer)

d_layer = Convolution2D(128, 3, 3, border_mode='same')(d_layer)
d_layer = LeakyReLU()(d_layer)
d_layer = MaxPooling2D(pool_size=(2, 2))(d_layer) # 4 -> 2
d_layer = SpatialDropout2D(0.25)(d_layer)

d_layer = Flatten()(d_layer) # 2 x 2 x 128 -> 512

d_layer = Dense(256)(d_layer)
d_layer = LeakyReLU()(d_layer)
d_layer = Dropout(0.5)(d_layer)

d_layer = Dense(256)(d_layer)
d_layer = LeakyReLU()(d_layer)
d_layer = Dropout(0.5)(d_layer)

d_layer = Dense(10)(d_layer)
d_output = Activation('softmax')(d_layer)

discriminator = Model(d_input, d_output)
discriminator.compile(loss='categorical_crossentropy', optimizer=disc_optimizer, metrics=['categorical_accuracy'])
discriminator.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_1 (InputLayer)             (None, 32, 32, 3)     0                                            
____________________________________________________________________________________________________
convolution2d_1 (Convolution2D)  (None, 32, 32, 32)    896         input_1[0][0]                    
____________________________________________________________________________________________________
leakyrelu_1 (LeakyReLU)          (None, 32, 32, 32)    0           convolution2d_1[0][0]            
____________________________________________________________________________________________________
convolution2d_2 (Convolution2D)  (None, 32, 32, 32)    9248        leakyrelu_1[0][0]                
___________________________________________________________________________________________

In [5]:
discriminator.fit(X_train, y_train,
              batch_size=disc_batch_size,
              nb_epoch=nb_epoch, 
              verbose=2, 
              validation_split=0.1)

Train on 45000 samples, validate on 5000 samples
Epoch 1/200
10s - loss: 2.1266 - categorical_accuracy: 0.2039 - val_loss: 1.7372 - val_categorical_accuracy: 0.3564
Epoch 2/200
9s - loss: 1.7446 - categorical_accuracy: 0.3587 - val_loss: 1.5004 - val_categorical_accuracy: 0.4488
Epoch 3/200
9s - loss: 1.5465 - categorical_accuracy: 0.4381 - val_loss: 1.3272 - val_categorical_accuracy: 0.5206
Epoch 4/200
9s - loss: 1.3927 - categorical_accuracy: 0.5023 - val_loss: 1.1583 - val_categorical_accuracy: 0.5842
Epoch 5/200
9s - loss: 1.2724 - categorical_accuracy: 0.5534 - val_loss: 1.0624 - val_categorical_accuracy: 0.6314
Epoch 6/200
9s - loss: 1.1689 - categorical_accuracy: 0.5926 - val_loss: 0.9659 - val_categorical_accuracy: 0.6662
Epoch 7/200
9s - loss: 1.0829 - categorical_accuracy: 0.6260 - val_loss: 0.9250 - val_categorical_accuracy: 0.6882
Epoch 8/200
9s - loss: 1.0188 - categorical_accuracy: 0.6520 - val_loss: 0.8487 - val_categorical_accuracy: 0.7062
Epoch 9/200
9s - loss: 0.9592 

In [6]:
score = discriminator.evaluate(X_test, y_test)

print "Test loss:", score[0]
print "Test accuracy:", score[1]

10000/10000 [==============================] - 2s     